In [1]:
import torch
import gpytorch
import numpy as np

from bo_functions import calc_ei_basic
from bo_functions import create_y_data_basic
from bo_functions import ExactGPModel
from bo_functions import calc_GP_parameters_basic

from bo_plotters import y_plotter_basic
from bo_plotters import stdev_plotter_basic
from bo_plotters import ei_plotter_basic

In [2]:
# Training data is 10^2 points in [-2,2] inclusive regularly spaced
Theta1 = np.linspace(-2,2,10) #1x10
Theta2 = np.linspace(-2,2,10) #1x10
x = torch.tensor(np.linspace(-2,2,5)) #1x5

#Creates a mesh for training data
train_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform Arrays 10x10 (.T turns this into 10 10x2 arrays)

#Lists every combination of training Theta1 and Theta2
train_T = torch.tensor(train_mesh.T.reshape(-1, 2))#100x2

#Set noise parameters and true value of Theta to generate training data
noise_std = 0.1**2
Theta_True = torch.tensor([1,-1]) #1x2
train_y = create_y_data_basic(Theta_True, train_T, x, noise_std)

NameError: name 'torch' is not defined

In [ ]:
# initialize likelihood and model
##Assumes a homoskedastic noise model p(y | f) = f + noise
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# We will use the simplest form of GP model, exact inference
#Defines our model in terms of the class parameters in bo_functions
model = ExactGPModel(train_T, train_y, likelihood)

In [ ]:
# Find optimal model hyperparameters
training_iter = 300

#Puts the model in training mode
model.train()

#Puts the likelihood in training mode
likelihood.train()

# Use the adam optimizer
    #algorithm for first-order gradient-based optimization of stochastic objective functions
    # The method is also appropriate for non-stationary objectives and problems with very noisy and/or sparse gradients. 
    #The hyper-parameters have intuitive interpretations and typically require little tuning.
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  #Needs GaussianLikelihood parameters, and a learning rate
    #lr default is 0.001

# Calculate"Loss" for GPs

#The marginal log likelihood (the evidence: quantifies joint probability of the data under the prior)
#returns an exact MLL for an exact Gaussian process with Gaussian likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model) #Takes a Gaussian likelihood and a model, a bound Method
#iterates a give number of times
for i in range(training_iter): #0-299
    # Zero gradients from previous iteration - Prevents past gradients from influencing the next iteration
    optimizer.zero_grad() 
    # Output from model
    output = model(train_T) # A multivariate norm of a 1 x 100 tensor
    # Calc loss and backprop gradients
    #Minimizing -logMLL lets us fit hyperparameters
    loss = -mll(output, train_y) #A number (tensor)
    #computes dloss/dx for every parameter x which has requires_grad=True. 
    #These are accumulated into x.grad for every parameter x
    loss.backward()
#     print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#         i + 1, training_iter, loss.item(),
#         model.covar_module.base_kernel.lengthscale.item(),
#          model.likelihood.noise.item()
#     ))
    #optimizer.step updates the value of x using the gradient x.grad. For example, the SGD optimizer performs:
    #x += -lr * x.grad
    optimizer.step()

In [ ]:
# Get into evaluation (predictive posterior) mode
#Puts model in evaluation mode
model.eval()
#Puts likelihood in evaluation mode
likelihood.eval()

#Define Testing Data
test_Theta1 =  np.linspace(-1,1,5) #1x5
test_Theta2 =  np.linspace(-1,1,5) #1x5
test_mesh = np.array(np.meshgrid(test_Theta1, test_Theta2)) #2 Uniform 5x5 arrays
test_T = torch.tensor(test_mesh.T.reshape(-1, 2)) #25 x 2

In [ ]:
GP_Output = calc_GP_parameters_basic(model, likelihood, test_T)

model_mean = GP_Output[0]
model_variance = GP_Output[1]
model_stdev = GP_Output[2]
model_sse = GP_Output[3]

#Finds the index where sse is the smallest and finds which Theta combination corresponds to that value
Theta_Opt_GP = test_T[np.argmin(model_sse)].numpy() #1x2
print("The GP predicts that Theta1 =",Theta_Opt_GP[0],"and Theta2 =", Theta_Opt_GP[1])

#calculates best_error and expected improvement
best_error = np.argmax(model_sse)
ei = calc_ei_basic(best_error,model_mean,model_variance)
print(ei)

#Formats ei points into a suitable graphing form    
ei_map = ei.reshape(len(test_Theta1),-1) # 5x5
#Formats sse data points into a suitable graphing form    
sse_map = model_sse.reshape(len(test_Theta1),-1) #5 x 5
#Formats stdev data points into suitable graphing form
stdev_map = model_stdev.reshape(len(test_Theta1),-1) #5 x 5

In [ ]:
title = "SSE"
y_plotter_basic(test_mesh, sse_map, Theta_True, Theta_Opt_GP, title)

In [ ]:
stdev_plotter_basic(test_mesh, stdev_map,Theta_True, Theta_Opt_GP)

In [ ]:
ei_plotter_basic(test_mesh, ei_map, Theta_True, Theta_Opt_GP)